# NEXRAD Level 2 Parser Verification - PyART
This notebook tests the PyART NEXRAD Level 2 parser and outputs results for comparison

In [4]:
%pip install boto3

  Using cached boto3-1.40.45-py3-none-any.whl.metadata (6.7 kB)
  Using cached botocore-1.40.45-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.14.0-py3-none-any.whl.metadata (1.7 kB)
   ---------------------------------------- 0.0/14.1 MB ? eta -:--:--
   --------- ------------------------------ 3.4/14.1 MB 16.8 MB/s eta 0:00:01
   ----------------------- ---------------- 8.4/14.1 MB 20.8 MB/s eta 0:00:01
   ----------------------------------- ---- 12.3/14.1 MB 22.0 MB/s eta 0:00:01
   ---------------------------------------  13.9/14.1 MB 20.8 MB/s eta 0:00:01
   ---------------------------------------- 14.1/14.1 MB 14.5 MB/s  0:00:00

  Attempting uninstall: botocore

    Found existing installation: botocore 1.40.18

   ---------------------------------------- 0/3 [botocore]
   ---------------------------------------- 0/3 [botocore]
   ---------------------------------------- 0/3 [botocore]
    Uninstalling botocore-1.40.18:
   ----------------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.24.2 requires botocore<1.40.19,>=1.40.15, but you have botocore 1.40.45 which is incompatible.


In [12]:
import pyart
from pyart.io.nexrad_level2 import NEXRADLevel2File
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import json
import numpy as np
from datetime import datetime

In [13]:
# Download a sample NEXRAD file from S3
# Using: 2013/05/06/KABR (site that appeared in the AWS CLI example)

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket = 'unidata-nexrad-level2'
prefix = '2013/05/06/KABR/'

# List files for this date/site
response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix, MaxKeys=1)

if 'Contents' in response and len(response['Contents']) > 0:
    # Get the first file
    file_key = response['Contents'][0]['Key']
    # Extract filename from the key
    remote_filename = file_key.split('/')[-1]
    local_file = f'data/{remote_filename}'
    
    print(f'Downloading: {file_key}')
    s3.download_file(bucket, file_key, local_file)
    print(f'Downloaded to: {local_file}')
    print(f'File key saved for Node.js test: {file_key}')
    
    # Save file info for Node.js
    with open('data/test_file_info.json', 'w') as f:
        json.dump({'key': file_key, 'local_path': local_file}, f, indent=2)
else:
    print('No files found for this date/site')

Downloading: 2013/05/06/KABR/KABR20130506_000526_V06.gz
Downloaded to: data/KABR20130506_000526_V06.gz
File key saved for Node.js test: 2013/05/06/KABR/KABR20130506_000526_V06.gz


In [15]:
# Use prepare_for_read to decompress and open the file, then parse with NEXRADLevel2File
from pyart.io.common import prepare_for_read

# Prepare the file for reading (handles decompression)
file_handle = prepare_for_read(local_file, storage_options={})

# Parse using NEXRADLevel2File directly
nexrad_file = NEXRADLevel2File(file_handle)

print('File parsed successfully with NEXRADLevel2File!')
print(f'Volume header info:')
print(f'  - Volume number: {nexrad_file.volume_header}')
print(f'  - Number of message_31 records: {len(nexrad_file.msg31_data_block_headers) if hasattr(nexrad_file, "msg31_data_block_headers") else "N/A"}')

File parsed successfully with NEXRADLevel2File!
Volume header info:
  - Volume number: {'tape': b'AR2V0006.', 'extension': b'843', 'date': 15832, 'time': 329000, 'icao': b'KABR'}
  - Number of message_31 records: N/A


In [16]:
# Extract data using NEXRADLevel2File class methods
output = {
    'volume_header': nexrad_file.volume_header,
    'vcp': nexrad_file.vcp,
    'nscans': nexrad_file.nscans,
    'location': nexrad_file.location(),
    'vcp_pattern': nexrad_file.get_vcp_pattern(),
}

# Get scan information for all scans
output['scan_info'] = nexrad_file.scan_info()

# Get times, angles, and velocities for all scans
output['times'] = nexrad_file.get_times()
output['azimuth_angles'] = nexrad_file.get_azimuth_angles()
output['elevation_angles'] = nexrad_file.get_elevation_angles()
output['target_angles'] = nexrad_file.get_target_angles()
output['nyquist_vel'] = nexrad_file.get_nyquist_vel()
output['unambiguous_range'] = nexrad_file.get_unambigous_range()

# Get number of rays for each scan
output['nrays_per_scan'] = {}
for scan in range(nexrad_file.nscans):
    output['nrays_per_scan'][f'scan_{scan}'] = nexrad_file.get_nrays(scan)

print('Data extraction complete!')
print(json.dumps(output, indent=2, default=str))

Data extraction complete!
{
  "volume_header": {
    "tape": "b'AR2V0006.'",
    "extension": "b'843'",
    "date": 15832,
    "time": 329000,
    "icao": "b'KABR'"
  },
  "vcp": {
    "header": {
      "size": 180,
      "channels": 8,
      "type": 5,
      "seq_id": 39315,
      "date": 15832,
      "ms": 326622,
      "segments": 1,
      "seg_num": 1
    },
    "msg5_header": {
      "msg_size": 172,
      "pattern_type": 2,
      "pattern_number": 32,
      "num_cuts": 7,
      "clutter_map_group": 1,
      "doppler_vel_res": 2,
      "pulse_width": 2,
      "spare": "b'\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'"
    },
    "cut_parameters": [
      {
        "elevation_angle": 88,
        "channel_config": 0,
        "waveform_type": 1,
        "super_resolution": 11,
        "prf_number": 1,
        "prf_pulse_count": 64,
        "azimuth_rate": 3616,
        "ref_thresh": 4,
        "vel_thresh": 4,
        "sw_thresh": 4,
        "zdr_thres": 4,
        "phi_thres": 

In [17]:
# Get available moments and their data
# Common NEXRAD moments: REF (reflectivity), VEL (velocity), SW (spectrum width), ZDR, PHI, RHO

moments_to_check = ['REF', 'VEL', 'SW', 'ZDR', 'PHI', 'RHO']
output['moments'] = {}

for moment in moments_to_check:
    try:
        # Get range information for first scan
        range_info = nexrad_file.get_range(0, moment)
        # Get data for this moment (using max_ngates from range info)
        max_ngates = len(range_info)
        data = nexrad_file.get_data(moment, max_ngates)
        
        output['moments'][moment] = {
            'available': True,
            'range_info': range_info.tolist() if hasattr(range_info, 'tolist') else list(range_info),
            'max_ngates': max_ngates,
            'data_shape': data.shape,
            'data_sample': data[0, :10].tolist() if len(data.shape) > 1 else []
        }
        print(f'Moment {moment}: Available, shape {data.shape}')
    except Exception as e:
        output['moments'][moment] = {'available': False, 'error': str(e)}
        print(f'Moment {moment}: Not available - {str(e)}')

Moment REF: Available, shape (3960, 1832)
Moment VEL: Not available - 'VEL'
Moment SW: Not available - 'SW'
Moment ZDR: Available, shape (3960, 1192)
Moment PHI: Available, shape (3960, 1192)
Moment RHO: Available, shape (3960, 1192)


In [ ]:
# Save output to file
with open('output/nexrad_level2_file_output.json', 'w') as f:
    json.dump(output, f, indent=2, default=str)

print('Output saved to: output/nexrad_level2_file_output.json')

Output saved to: output/nexrad_level2_file_output.json


In [ ]:
# Summary
print(f"\n=== NEXRADLevel2File Summary ===")
print(f"Location: {output['location']}")
print(f"VCP Pattern: {output['vcp_pattern']}")
print(f"Number of scans: {output['nscans']}")
print(f"Available moments: {[m for m, info in output['moments'].items() if info.get('available', False)]}")
print(f"\nAnalysis complete!")